# Handwriting recognition (HWR)

### Author: Mohammed Shehab

This script provides a comprehensive guide to:
1. Loading and preprocessing the dataset
2. Constructing a Convolutional Neural Network (CNN) model architecture
3. Performing hyperparameter tuning to optimize the CNN model
4. Exporting the best-performing model for deployment

**Deployment will be facilitated using Docker containerization to ensure a scalable and reproducible environment.**


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras_tuner import HyperModel, RandomSearch
import numpy as np

Check GPU availablilty 

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Load dataset

In [4]:
# Load and preprocess the MNIST dataset
def load_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    return (x_train, y_train), (x_test, y_test)

### Define a HyperModel class for Keras Tuner with adjustable hyperparameters

In [5]:
class CNNHyperModel(HyperModel):
    def build(self, hp):
        model = models.Sequential()
        
        # Conv1
        model.add(layers.Conv2D(
            filters=hp.Int('conv1_filters', min_value=16, max_value=64, step=16),
            kernel_size=(3, 3),
            activation='relu',
            input_shape=(28, 28, 1)
        ))
        model.add(layers.MaxPooling2D((2, 2)))
        
        # Conv2_1 and Conv2_2
        for i in range(2):
            model.add(layers.Conv2D(
                filters=hp.Int(f'conv2_filters_{i+1}', min_value=32, max_value=128, step=32),
                kernel_size=(3, 3),
                activation='relu'
            ))
        
        # MaxPooling
        model.add(layers.MaxPooling2D((2, 2)))
        
        # Conv3_1 and Conv3_2
        model.add(layers.Conv2D(
            filters=hp.Int('conv3_filters', min_value=128, max_value=512, step=128),
            kernel_size=(3, 3),
            activation='relu'
        ))
        
        model.add(layers.Flatten())
        
        # Fully Connected Layer 1
        model.add(layers.Dense(
            units=hp.Int('fc1_units', min_value=500, max_value=1000, step=250),
            activation='relu'
        ))
        
        # Fully Connected Layer 2
        model.add(layers.Dense(
            units=hp.Int('fc2_units', min_value=250, max_value=500, step=250),
            activation='relu'
        ))
        
        # Output Layer
        model.add(layers.Dense(10, activation='softmax'))
        
        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            ),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model

In [6]:
(x_train, y_train), (x_test, y_test) = load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [10]:
x_train.shape

(60000, 28, 28, 1)

In [11]:
x_test.shape

(10000, 28, 28, 1)

### Initialize the Keras Tuner

In [12]:
tuner = RandomSearch(
    CNNHyperModel(),
    objective='val_accuracy',
    max_trials=10,  # Number of different hyperparameter combinations to try
    executions_per_trial=2,  # Average results over multiple runs
    directory='tuning_results',
    project_name='mnist_cnn_tuning'
)

### Perform hyperparameter search

In [13]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 10 Complete [00h 02m 09s]
val_accuracy: 0.991349995136261

Best val_accuracy So Far: 0.9930500090122223
Total elapsed time: 00h 21m 18s


In [14]:
# Retrieve the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters found:")
print(best_hyperparameters.values)

Best hyperparameters found:
{'conv1_filters': 48, 'conv2_filters_1': 96, 'conv2_filters_2': 96, 'conv3_filters': 256, 'fc1_units': 1000, 'fc2_units': 250, 'learning_rate': 0.0003031375640856382}


### Export model for deployment phase

In [22]:
import os
from tensorflow.keras.models import save_model

model_version = "v1_0_0"  # Update this version number as needed
# Define the directory and model file paths separately
model_directory = './exported_models/'
model_file_path = os.path.join(model_directory, f'best_model_{model_version}.h5')

# Create the directory if it doesn't exist
os.makedirs(model_directory, exist_ok=True)
print(model_file_path)
# Save the model in the specified versioned file path
best_model.save(model_file_path)
print(f"Model saved to {model_file_path}")


./exported_models/best_model_v1_0_0.h5
Model saved to ./exported_models/best_model_v1_0_0.h5
